## What is Threading?

In [ ]:
+-------------------------------------+          +----------------------------------+
|       Single-Threaded Process       |          |      Multi-Threaded Process      |
|-------------------------------------|          |----------------------------------|
|  +------------+                     |          |  +------------+  +------------+  |
|  |  Thread 1  |                     |          |  |  Thread 1  |  |  Thread 2  |  |
|  +------------+                     |          |  +------------+  +------------+  |
|  |  Task 1    |                     |          |  |  Task A    |  |  Task C    |  |
|  |  Task 2    |                     |          |  |  Task B    |  |  Task D    |  |
|  |  Task 3    |                     |          |  +------------+  +------------+  |
|  +------------+                     |          |  +------------+  +------------+  |
+-------------------------------------+          |  |  Thread 3  |  |  Thread 4  |  |
                                                 |  +------------+  +------------+  |
                                                 |  |  Task E    |  |  Task G    |  |
                                                 |  |  Task F    |  |  Task H    |  |
                                                 |  +------------+  +------------+  |
                                                 +----------------------------------+

**Single Thread:** Default; main thread executes code sequentially.

**Multithreading:** Needed for concurrent/simultaneous code execution.

## What is a thread?

- Threads are basic CPU units.
- Single process contains multiple threads.
- Threads share code, data, files.
- Each thread has its own registers, separate stack.

## Why or When Threading?

**App Screen Persistence:**

***Static Image Display:***
```python
# Main Thread
while(True):
    displayScreen()
```
Shows fleeting images; appears like the app is running but isn't.

***Heavy Operation + Display:***
```python
# Main Thread
while(True):
    # Heavy Operation
    displayScreen()
```
Results in flicker; heavy op delays screen display.

***Network Call + Display:***
```python
# Main Thread
while(True):
    Image = request(ImageUrl)
    displayScreen()
```
Causes screen to pop up based on network delay.

***Solution: Multithreading***
```python
# Main Thread
Image = None

def startAnotherThread():
    while(True):
        Image = request(ImageUrl)

while(True):
    displayScreen(Image)
```
`startAnotherThread()` fetches images concurrently; `displayScreen()` shows available images.

In [3]:
# another example ---> multithreading in servers is used to handle multiple requests. 
#                      mechanism involves diff threads for each request

## How threads are handled by OS?

**Threads Running Concurrently** a **"False Statement"**. Threads are not truly running concurrently.

**Reality:** Limited by single-core CPUs; **Concurrent** ≠ **Parallel**

In [ ]:
                Web Server                         Thread
+-----------------------------------------+          ^           +--------------------------------+
|       +-------------------------+       |          |  t3 ----> | Running  | Sleeping | Running  |
|       |     Request Handler     |       |          |           +--------------------------------+
|       +-------------------------+       |          |           +--------------------------------+
|  +---------+  +---------+  +---------+  |          |  t2 ----> | Sleeping | Running  | Sleeping |
|  | Client1 |  | Client2 |  | Client3 |  |          |           +--------------------------------+
|  +---------+  +---------+  +---------+  |          |           +--------------------------------+
+-----------------------------------------+          |  t1 ----> | Running  | Running  | Sleeping |
                                                     |           +--------------------------------+  
                                                     +-----------------------------------------------> Time

3 threads.

CPU switches between them.

Only 1 thread executes at a time.

Effect: Concurrency.

## Implementation

In [6]:
from time import sleep, time
import threading

start = time()

def task(id):
    print(f"Sleeping...{id}")
    sleep(1)
    print(f"Woke up...{id}")

threads = [threading.Thread(target=task, args=[i]) for i in range(10)]
for t in threads:
    t.start()

for t in threads:
    t.join()

end = time()
print(f"Main Thread Duration: {end - start} sec")

Sleeping...0
Sleeping...1
Sleeping...2
Sleeping...3
Sleeping...4
Sleeping...5
Sleeping...6
Sleeping...7
Sleeping...8
Sleeping...9
Woke up...8Woke up...7
Woke up...5
Woke up...4
Woke up...2
Woke up...1
Woke up...9
Woke up...3
Woke up...6
Woke up...0

Main Thread Duration: 1.0146267414093018 sec


## Thread Synchronization

In [10]:
import threading

balance = 200
lock = threading.Lock()

def deposit(amount, times, lock):
    global balance
    for _ in range(times):
        lock.acquire()
        balance += amount
        lock.release()

def withdraw(amount, times, lock):
    global balance
    for _ in range(times):
        lock.acquire()
        balance -= amount
        lock.release()

deposit_thread = threading.Thread(target=deposit, args=[1, 100000, lock])
withdraw_thread = threading.Thread(target=withdraw, args=[1, 100000, lock])

deposit_thread.start()
withdraw_thread.start()
deposit_thread.join()
withdraw_thread.join()

print(balance)

200
